In [ ]:
# !pip install ydata-profiling
# !pip install ipywidgets
# !pip install seaborn
# !pip install hvplot
# !pip install ipympl
# !pip install plotly
# !pip install jupyter_bokeh
# !pip install shap
# !pip install dash dash-bootstrap-components
# !pip install dask
# !pip install dask[distributed]
# !pip install pyarrow

In [ ]:
%cd ..

In [ ]:
data_path = "data/Quote_and_Order_Data_2025_01_30.csv"

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from gomat_markup_opt.core.train_conversion_probability_model import (
    TOP_10_REGIONS,
    CUSTOMER_COLUMN,
    ORDER_DATE,
    ORDER_ID_COLUMN,
    PRODUCT_ID_COLUMN,
    PLANT_CATEGORY_COLUMN,
    PLANT_NAME_COLUMN,
    QUOTE_STATUS,
    REGION_COLUMN,
    SELLER_PRICE,
    SELLER_PRICE_RELATIVE_TO_MARKET,
    SELLER_TOTAL,
    SELECTED_COLUMNS,
    MARKUP_COLUMN,
    TOTAL_ITEMS_COLUMN,
    REQUEST_PURPOSE_COLUMN,
    QTO_COLUMN,
)

In [ ]:
data = pd.read_csv(
    data_path,
    delimiter=",",
    encoding="latin1",
    on_bad_lines="warn",
    engine="python",
)

In [ ]:
SELLER_PRICE_BINS = "Seller Total Bins"

In [ ]:
PLANT_REGION_COLS = [
    REGION_COLUMN,
    PLANT_NAME_COLUMN,
]

In [ ]:
GROUP_BY_COLS = [
        QTO_COLUMN,
        REQUEST_PURPOSE_COLUMN,
        REGION_COLUMN,

        PLANT_NAME_COLUMN, 
        SELLER_PRICE_BINS
        ]

In [ ]:
import joblib
from gomat_markup_opt.core.train_optimization_space_estimator import MarkupOptSpaceEstimator, QuantileBinClassifier, Pipeline, QuantileLevels

In [ ]:
MODELS_FILEPATH = "dev/markup_optimization_interval"

model_path = f"{MODELS_FILEPATH}/estimator.pkl"
interval_estimator = joblib.load(model_path)
interval_estimator

In [ ]:
MODELS_FILEPATH = "dev/model_files_march_10_no_augmentation"
model_path = f"{MODELS_FILEPATH}/boosting_model.pkl"
conversion_estimator = joblib.load(model_path)
conversion_estimator

In [ ]:
# select quotes of january 2025
START = "2025-01-01"
END = "2025-01-31"
data = data[
    pd.to_datetime(
        data[ORDER_DATE], dayfirst=True
        ).between(START, END)]

In [ ]:
from gomat_markup_opt.core.train_conversion_probability_model import _prepare_input_data, _feature_engineering_median_price

prepared_data = _prepare_input_data(data)


selected_df = prepared_data
selected_df = _feature_engineering_median_price(selected_df)

# LOAD SAVED FILE

selected_df.head()

In [ ]:
len(selected_df)

In [ ]:
processed_df = interval_estimator.predict(selected_df)
len(processed_df)

In [ ]:
HITTED_BOUNDS = 0

def search_and_optimize(item_infos):
    global HITTED_BOUNDS

    NUM_TEST = 50
    optimization_interval_bounds= item_infos[["low", "high"]].values.astype(np.float32)
    optimization_interval = np.linspace(*tuple(optimization_interval_bounds), num=NUM_TEST)
    # optimization_interval = np.arange(*tuple(optimization_interval), step=0.1)
    
    simulated_items = pd.concat([item_infos] * NUM_TEST, axis=1).T.reset_index(drop=True)
    simulated_items[MARKUP_COLUMN] = optimization_interval
    simulated_items["conversion_probs"] = conversion_estimator.predict_proba(simulated_items[SELECTED_COLUMNS])[:,1]
    
    simulated_items["profit"] = simulated_items["conversion_probs"] * simulated_items[MARKUP_COLUMN]

    best_markup_idx = simulated_items["profit"].idxmax()
    best_markup = simulated_items[MARKUP_COLUMN].iloc[best_markup_idx]
    
    # if we hit the bounds
    if np.isclose(best_markup, optimization_interval_bounds, atol=1e-2).any():
        HITTED_BOUNDS += 1
        
    return best_markup

In [ ]:
# processed_df.sample(10).apply(search_and_optimize, axis=1)

In [ ]:
import dask
import dask.dataframe as dd
from tqdm.dask import TqdmCallback

processed_df[["high", "low"]] = processed_df[["high", "low"]].astype(float)

processed_dd = dd.from_pandas(processed_df, npartitions=48)

find_optimal_markups_task = processed_dd.apply(search_and_optimize, axis=1, meta=(None, 'f8'))

with TqdmCallback():
    (optimal_markups,) = dask.compute(find_optimal_markups_task)


# res = processed_dd.map_partitions(lambda df: df.apply(search_and_optimize, axis=1), meta=(None, 'f8')).compute()
# res = processed_dd.apply(search_and_optimize, axis=1, meta=(None, 'f8')).compute()

In [ ]:
optimal_markups.name = "optimal_markup"
optimal_markups.head()

In [ ]:
processed_df = processed_df.join(optimal_markups)
processed_df.head(1)

In [ ]:
# from tqdm import tqdm
# tqdm.pandas()

# processed_df["best_markup"] = processed_df.progress_apply(search_and_optimize, axis=1)

In [ ]:
HITTED_BOUNDS

In [ ]:
processed_df["Markup"].hist()

In [ ]:
processed_df["optimal_markup"].clip(0.0, 71.0).hist()

In [ ]:
accepted_quotes = processed_df[processed_df[QUOTE_STATUS] == 1]
accepted_quotes.head(1)

In [ ]:
(accepted_quotes["Markup"] * accepted_quotes[SELLER_TOTAL]).sum().round()

In [ ]:
(accepted_quotes["optimal_markup"] * accepted_quotes[SELLER_TOTAL]).sum().round()

In [ ]:
(accepted_quotes["optimal_markup"] - accepted_quotes["Markup"]).mean()